In [5]:
#! export OPENBLAS_NUM_THREADS=4 # this is for parrallel cpu running 
import numpy as np
import matplotlib.pyplot as plt
import sys
import caffe
import cv2
net = caffe.Net('/home/saadi/caffe_ssd/models/AgeGender/IMDB/Age-finetuned/age.prototxt',
                '/home/saadi/caffe_ssd/models/AgeGender/IMDB/Age-finetuned/dex_chalearn_iccv2015.caffemodel',
                caffe.TEST)
net1 = caffe.Net('/home/saadi/caffe_ssd/models/AgeGender/IMDB/Gender/gender.prototxt',
                '/home/saadi/caffe_ssd/models/AgeGender/IMDB/Gender/gender.caffemodel',
                caffe.TEST)

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [8]:
cap = cv2.VideoCapture(0)
#################################################################################################
width=cap.get(cv2.CAP_PROP_FRAME_WIDTH)
print width
height=cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
print height

cap.set(cv2.CAP_PROP_FRAME_WIDTH,1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,1024)

width=cap.get(cv2.CAP_PROP_FRAME_WIDTH)
print width
height=cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
print height
framerate=cap.get(cv2.CAP_PROP_FPS)
print ('framerate is =',framerate)
#################################################################################################
#cv2.waitKey(0)#proceed to remaining code when you press a key
#
p = 'text'
s = 'text'
font = cv2.FONT_HERSHEY_SIMPLEX
framerate=30
frm_rate=0
r='calcuating....'
gen='claculating...'

while(cap.isOpened()):
# while 1:
    ret, frame = cap.read()
    #
    #cv2.waitKey(0)#proceed to remaining code when you press a key
    #
    frm_rate=frm_rate+1
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        #cv2.imshow('frame',frame)
        transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
        transformer.set_mean('data', np.load('python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1))
        transformer.set_transpose('data', (2,0,1))
        transformer.set_channel_swap('data', (2,1,0)) # if using RGB instead of BGR
        transformer.set_raw_scale('data', 255.0)
        net.blobs['data'].reshape(1,3,224,224)
        
        
        ##############################################################################
        transformer1 = caffe.io.Transformer({'data': net1.blobs['data'].data.shape})
        transformer1.set_mean('data', np.load('python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1))
        transformer1.set_transpose('data', (2,0,1))
        transformer1.set_channel_swap('data', (2,1,0)) # if using RGB instead of BGR
        transformer1.set_raw_scale('data', 255.0)
        net1.blobs['data'].reshape(1,3,224,224)
        
        
        if frm_rate >= framerate : 
         frm_rate=0
         net1.blobs['data'].data[0,...] = transformer1.preprocess('data', frame)
         net.blobs['data'].data[0,...] = transformer.preprocess('data', frame)
         output1 = net1.forward()
         output = net.forward()
         p = output1['prob'].argmax()
        #p_str=p.tolist()
         print p
         
        ##############################################################################
         s = output['prob'].argmax() 
        #p_str=p.tolist()
        # print type(x)
         r = s.astype('str')
       # print 'age retested'
        age = 'Age= '+ r
        if p==0:
          gen = 'Female' 
        else  :
          gen = 'Male'
    
        
    cv2.putText(frame, gen,(x+w,y), font,0.75,(0,255,255),2)
    cv2.putText(frame, age,(x+w,y+h), font,0.75,(255,0,255),2)
    cv2.imshow('frame',frame)
        #print output['prob'].argmax()
        
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

 
cap.release()
cv2.destroyAllWindows()
        

0.0
0.0
0.0
0.0
('framerate is =', 0.0)


error: /io/opencv/modules/highgui/src/window.cpp:539: error: (-2) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Carbon support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function cvDestroyAllWindows
